In [1]:
import scipy.io as sio
import scipy.sparse as sp
import random
import torch
import pickle
from torch_geometric.utils.convert import to_scipy_sparse_matrix
import numpy as np

In [8]:
data3 = sio.loadmat("./dataset2/{}.mat".format('yelp'))
data3

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Fri Jun 16 12:06:31 2023',
 '__version__': '1.0',
 '__globals__': [],
 'adj': <45954x45954 sparse matrix of type '<class 'numpy.int64'>'
 	with 7693958 stored elements in Compressed Sparse Column format>,
 'feat': array([[0.02237555, 0.07049485, 0.42868164, ..., 0.5920398 , 0.13930348,
         0.49751243],
        [0.02492766, 0.99998516, 0.99998516, ..., 0.5920398 , 0.13930348,
         0.49751243],
        [0.00617256, 0.07049485, 0.42868164, ..., 0.5920398 , 0.13930348,
         0.49751243],
        ...,
        [0.00908079, 0.35002595, 0.42868164, ..., 0.48258707, 0.80099505,
         0.1641791 ],
        [0.00617256, 0.07049485, 0.99998516, ..., 0.44278607, 0.4477612 ,
         0.5870647 ],
        [0.00317531, 0.35002595, 0.42868164, ..., 0.44278607, 0.4477612 ,
         0.5870647 ]], dtype=float32),
 'clabels': array([], shape=(0, 0), dtype=float64),
 'alabels': array([[0, 0, 0, ..., 0, 0, 0]])}

In [9]:
data2 = sio.loadmat("./dataset2/{}.mat".format('Automotive'))
data2

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Fri Jun 16 11:58:58 2023',
 '__version__': '1.0',
 '__globals__': [],
 'adj': <28922x28922 sparse matrix of type '<class 'numpy.int64'>'
 	with 213300 stored elements in Compressed Sparse Column format>,
 'feat': array([[0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 1., 1., 1.],
        [0., 0., 0., ..., 0., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]], dtype=float32),
 'clabels': array([], shape=(0, 0), dtype=float64),
 'alabels': array([[0, 0, 0, ..., 1, 1, 1]])}

In [4]:
data = sio.loadmat("./dataset2/{}.mat".format('citeseer'))
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Fri Jun 16 11:45:18 2023',
 '__version__': '1.0',
 '__globals__': [],
 'adj': <3327x3327 sparse matrix of type '<class 'numpy.float64'>'
 	with 10275 stored elements in Compressed Sparse Column format>,
 'feat': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'clabels': array([[3, 1, 5, ..., 3, 1, 5]], dtype=uint8),
 'alabels': array([[0, 0, 0, ..., 0, 0, 1]], dtype=uint8)}

In [27]:
data2['alabels'][0].astype(np.int64)

array([0, 0, 0, ..., 1, 1, 1])

In [33]:
feat = data2['feat']
feat

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [34]:
from sklearn.cluster import KMeans

k = 5
model = KMeans(n_clusters=k)
model.fit(feat)
labels_list = model.labels_

In [36]:
model.labels_.astype(np.int64)

array([3, 0, 1, ..., 2, 2, 2])

In [38]:
nodes = [[i for i in range(len(labels_list)) if labels_list[i]==j] for j in range(k)]
distances = [[np.linalg.norm(feat[i] - model.cluster_centers_[labels_list[i]]) for i in range(len(labels_list)) if labels_list[i]==j] for j in range(k)]
idx_known = np.hstack(np.array([[np.array(nodes[i])[np.array(distances[i]).argsort()][:min(len(distances[i]), 50)]] for i in range(k)], dtype=object).reshape(-1))

In [41]:
idx_known

array([15103, 18508, 16624, 20882, 20880, 13350, 13341, 17903, 17893,
       17892, 17901,  4616, 18475, 13321, 11621,  5024, 15543, 22546,
        3760, 16737, 14992, 17896, 11123, 21814, 15621, 20714, 20750,
       20727, 20728, 20753,  2707, 15230, 15231, 20887,  3177,  3176,
        3192, 20650, 11361, 20712, 20736, 22558, 15724, 22561, 22902,
       13313, 18450, 11118,  1699, 18424, 21268, 23272,   798, 22814,
        3917,   883,  8701, 14243, 25661, 25662, 20098, 10676, 10677,
       24241, 24242, 24304, 24305, 25485, 25486, 21109, 21110, 21111,
       13159,  2434, 14050, 14048, 24641, 25632, 25633, 24011, 24013,
       18807, 18805, 18806, 12912, 20923, 22616, 22615, 25199,  9732,
        9731, 22405, 23323, 21588,  1968, 18188, 24063, 24062, 23244,
       22540,  9400,   272,   960,  5817,  5820,   285,   128,  9536,
        5638,  5233,   728,   266,   717,  1229, 10336, 21532,   996,
        1801,  4422,  5413,  2846,    80,  7023,  3049,  6784,  3024,
        7548,  8004,